In [1]:
import pandas as pd
import re
from nltk import word_tokenize
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords
import string

In [2]:
import tashaphyne.arabic_const as arabconst

In [3]:
df1 = pd.read_csv('ar_dataset.csv')
df2 = pd.read_csv('L-HSAB', sep='\t')
df3 = pd.read_excel('TweetClassification-Summary.xlsx')
df4 = pd.read_excel('AJCommentsClassification-CF.xlsx')
lexicon = pd.read_csv('C:\\Users\\Solom\\Desktop\\hurtlex_AR.tsv', sep='\t')
lemmas = set(lexicon['lemma'])

In [4]:
old_lexicon = pd.read_csv('hurtlex_AR.tsv', sep='\t')

In [5]:
for idx,label in enumerate(df1['sentiment']):
    if 'hateful' in label:
        df1['sentiment'][idx] = 'hateful'
    elif 'offensive' in label:
        df1['sentiment'][idx] = 'offensive'
    elif 'abusive' in label:
        df1['sentiment'][idx] = 'hateful'
    elif 'fearful' in label:
        df1['sentiment'][idx] = 'hateful'
    elif 'disrespectful' in label:
        df1['sentiment'][idx] = 'hateful'

C:\Users\Solom\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Solom\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Solom\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/us

In [6]:
for idx,label in enumerate(df2['Class']):
    if 'hate' in label:
        df2['Class'][idx] = 'hateful'
    elif 'abusive' in label:
        df2['Class'][idx] = 'offensive'

In [7]:
for idx,label in enumerate(df3['aggregatedAnnotation']):
    if label == -2:
        df3['aggregatedAnnotation'][idx] = 'hateful'
    elif label == -1:
        df3['aggregatedAnnotation'][idx] = 'offensive'
    elif label == 0:
        df3['aggregatedAnnotation'][idx] = 'normal'

C:\Users\Solom\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Solom\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Solom\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [8]:
for idx, body in enumerate(df4['body']):
    if df4['languagecomment'][idx]== -1 or df4['languagecomment'][idx]==-2:
        df4['languagecomment'][idx] = 'hateful'
    elif df4['languagecomment'][idx]==0:
        df4['languagecomment'][idx] = 'normal'
            

C:\Users\Solom\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Solom\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [9]:
frames = [df1[['tweet', 'sentiment']], df2.rename(columns={"Tweet":"tweet","Class": "sentiment"}),
          df3[['text', 'aggregatedAnnotation']].rename(columns={"text":"tweet","aggregatedAnnotation": "sentiment"}),
         df4[['body', 'languagecomment']].rename(columns={"body":"tweet", "languagecomment":"sentiment"})]
df = pd.concat(frames, join='outer', ignore_index=True)

In [10]:
set(df['sentiment'])

{'hateful', 'normal', 'offensive'}

In [11]:
len(set(df['tweet']))

41881

In [12]:
def normalizeArabic(text):
    # Remove Tashkeel
    text = arabconst.HARAKAT_PAT.sub('', text)
    # Remove Repeated Characters
    text = re.sub(r'(.)\1+', r'\1', text)
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    return text

In [13]:
stop_words = [normalizeArabic(x) for x in stopwords.words('arabic')]
st = ISRIStemmer()

In [14]:
def remove_single_chars(text):
    words = text.split(" ")
    text = " ".join([word for word in words if len(word) > 1])
    return text

In [15]:
def clean(text):
    text = normalizeArabic(text)
    # Remove Punctuations
    arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
    english_punctuations = string.punctuation
    punctuations_list = arabic_punctuations + english_punctuations
    text = text.translate(str.maketrans('', '', punctuations_list))
    # Remove Hashtag Signs
    text = re.sub(r"#", " ", text)
    # Remove URLs, Mentions, Trailing Non-Whitespaces Characters
    text = re.sub(r"(?:\@|https?\://)\S+", " ", text)
    # Remove Numbers
    text = re.sub(r"\d+", " ", text)
    # Remove English Characters
    text = re.sub(r"[A-Z|a-z]+", " ", text)
    # Remove Single Characters
    text = remove_single_chars(text)
    # Remove Stop Words
    text = " ".join([word for word in word_tokenize(text) if not word in stop_words])
    return str(text)
    

In [16]:
def stem(text):
    # Stemming
    text = " ".join([st.stem(word)for word in word_tokenize(text)])
    return text

In [17]:
df['clean'] = df['tweet'].apply(lambda x: clean(x))

In [18]:
df['stemmed'] = df['clean'].apply(lambda x: stem(x))

In [19]:
lexicon['clean'] = lexicon['lemma'].apply(lambda x: clean(x))

In [20]:
lexicon['stem'] = lexicon['clean'].apply(lambda x: stem(x))

In [21]:
old_lexicon['clean'] = old_lexicon['lemma'].apply(lambda x: clean(x))
old_lexicon['stem'] = old_lexicon['clean'].apply(lambda x: stem(x))

In [23]:
df['tweet'][0]

'صلاة الفجر خير لك من ترديد بول البعير وسبي النساء واغتصاب طفلة نظافة ونشاط وحيوية #عقلانيون'

In [24]:
df['clean'][0]

'صلاه الفجر خير ترديد بول البعير وسبي النساء واغتصاب طفله نظافه ونشاط وحيويه عقلانيون'

In [31]:
nan_value = float("NaN")
old_lexicon.replace("", nan_value, inplace=True)
old_lexicon.dropna(subset = ["clean"], inplace=True)

In [33]:
old_lexicon.reset_index().head(8)

,index,id,pos,category,stereotype,lemma,level,clean,stem
0,0,AR1499,n,an,no,الأوروبية الآسيوية غريفون,inclusive,الاوروبيه الاسيويه غريفون,ورب سيه غرف
1,1,AR1048,n,cds,no,ديماجوجي,conservative,ديماجوجي,ديماجوجي
2,2,AR70,n,om,no,اللواطة,inclusive,الواطه,وطه
3,3,AR2743,n,dmc,yes,تشرد,inclusive,تشرد,شرد
4,4,AR1322,n,qas,no,الفلاح,inclusive,الفلاح,فلح
5,5,AR2310,n,an,no,دُودَة,inclusive,دوده,دود
6,6,AR931,n,an,no,التنصت,inclusive,التنصت,تنص
7,9,AR948,n,an,no,مغناج,conservative,مغناج,غنج


In [39]:
old_lexicon.reset_index().to_csv('old_lexicon.csv')

In [37]:
df.replace("", nan_value, inplace=True)
df.dropna(subset = ["clean"], inplace=True)
df.reset_index().to_csv('data.csv')

In [38]:
lexicon.replace("", nan_value, inplace=True)
lexicon.dropna(subset = ["clean"], inplace=True)
lexicon.reset_index().to_csv('New_Lexicon.csv')